# Clase: Fine-Tuning de Modelos con Python (2 horas) Objetivo general

Comprender el proceso de ajuste fino (fine-tuning) de un modelo preentrenado utilizando Python y librerías modernas, aplicando buenas prácticas de entrenamiento, evaluación y uso de recursos.

# Estructura de la clase (120 min)

Tiempo	Actividad	Descripción
0–10 min	Introducción al Fine-Tuning	Qué es el fine-tuning, diferencia con feature extraction, casos prácticos (NLP, visión, audio).
10–25 min	Conceptos clave	Modelos preentrenados, datasets, capas congeladas, transfer learning, overfitting y underfitting.
25–70 min	Demostración práctica en Notebook (Python)	Entrenar un modelo preentrenado usando transformers (Hugging Face) o torchvision.
70–90 min	Actividad guiada (hands-on)	Los estudiantes replican el notebook cambiando el dataset o las etiquetas.
90–110 min	Evaluación y métricas	Visualizar pérdidas, accuracy y ejemplos predichos.
110–120 min	Cierre y conclusiones	Preguntas, buenas prácticas, próximos pasos (modelos más grandes, optimización).

# Resultados esperados
- Comprender cómo usar un modelo preentrenado.
- Ajustarlo a un dataset específico.
- Evaluar su rendimiento.
- Probar predicciones con texto nuevo.

In [ ]:
# 1. Instalación y setup
# Estoy utilizando Python 3.10, ya que 3.13 es incompatible con Transformers 4
import sys
!{sys.executable} -m pip install -U transformers==4.44.2 tokenizers==0.19.1 accelerate==0.33.0 datasets==3.0.1 evaluate==0.4.2 sentencepiece

In [ ]:

# 1.5. FIX install transformers
import transformers
print(transformers.__version__)
print(transformers.__file__)

In [ ]:
# 2. Importar librerías
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np

In [14]:
# 3. Cargar dataset
from datasets import load_dataset
ds = load_dataset("imdb")
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
# 4. Cargar modelo preentrenado
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

In [ ]:
# 5. Tokenización
def tokenize_fn(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize_fn, batched=True)

In [24]:
!"c:\Users\Rodrigo\AppData\Local\Programs\Python\Python310\python.exe" -m pip install -U datasets==3.1.0 huggingface_hub==0.25.2

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1


You should consider upgrading via the 'c:\Users\Rodrigo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [27]:
import datasets, evaluate, transformers, huggingface_hub
print("datasets:", datasets.__version__)
print("evaluate:", evaluate.__version__)
print("transformers:", transformers.__version__)
print("huggingface_hub:", huggingface_hub.__version__)


datasets: 3.0.1
evaluate: 0.4.2
transformers: 4.44.2
huggingface_hub: 0.36.0


In [23]:
import sys
print(sys.executable)

c:\Users\Rodrigo\AppData\Local\Programs\Python\Python310\python.exe


In [28]:
import huggingface_hub
print(huggingface_hub.__version__)

0.36.0


In [19]:
import evaluate
accuracy = evaluate.load("accuracy")
accuracy.compute(predictions=[0,1], references=[0,1])

AttributeError: 'DownloadConfig' object has no attribute 'use_auth_token'

In [18]:
from transformers import TrainingArguments, Trainer, IntervalStrategy
import evaluate
import numpy as np

import os
os.environ["WANDB_DISABLED"] = "true"

# Cargar métrica correctamente
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # return accuracy.compute(predictions=predictions, references=labels)
    return accuracy.compute(predictions=[0,1], references=[0,1])

args = TrainingArguments(
    output_dir="./results",
    save_strategy="epoch",
    evaluation_strategy="epoch",  # ← parámetro correcto
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"].shuffle(seed=42).select(range(2000)),
    eval_dataset=tokenized["test"].shuffle(seed=42).select(range(500)),
    compute_metrics=compute_metrics,
)

trainer.train()


AttributeError: 'DownloadConfig' object has no attribute 'use_auth_token'

In [ ]:
# 7. Evaluación
trainer.evaluate()

In [ ]:
# 8. Prueba con texto nuevo
text = "This movie was absolutely amazing!"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
pred = outputs.logits.argmax().item()
print("Positiva" if pred == 1 else "Negativa")